# Summary - Top Down Approach (Memoization)
At each element, we have a yes or no decision; and once we make a yes, the next has to be a no

Then we can trace out all paths, and the path with the maximum value at the end is the answer

This would mean roughly $O(2^n)$ operations.

This can be coded up as a recursive call stack.

The base cases are when the input array is of length 1 (return itself), or 2 (return whichever is larger, which is an $O(1)$ operation).

Then we simply return `max(curr + rob(skip 2), rob(skip 1))`. So then the function will call itself over and over until we reach the case where there's only two elements left.

From here we observe that, `rob(skip 2)` and `rob(skip 1)` have a large overlap as we progress, since they are only off-set by 1.

So if we can cache `rob(...)` results, then we can save most of the operations, and shave the time complexity down to $O(n)$, in exchange of growing the space complexity to $O(n)$ in order to cache the subproblem results.

We can achieve so by building a `subproblem` nested function, and only pass in the indices then reference the original `nums` outside of the nested function. We cache the edge cases, and if edge cases are not encountered, also just enter the subproblem with the start index incremented by 1 (current not added) or by 2 (current is added).

## Time Complexity
$O(n)$ each `cache` index is calculated exactly once and `start` can go from `0` to `n - 1`

## Space Complexity
$O(n)$ for `cache` to store `n` entries, and the recursive stack can grow to the height of `n` to cover all indices before the `subproblem` recursive stack start having already cached values and returning the cached values.

In [ ]:
from typing import List


class SolutionBruteForce:
    def rob(self, nums: List[int]) -> int:
        if len(nums) == 1:
            return nums[0]

        if len(nums) == 2:
            return max(nums[0], nums[1])

        return max(nums[0] + self.rob(nums[2:]), self.rob(nums[1:]))

In [ ]:
from typing import List


class Solution:
    def rob(self, nums: List[int]) -> int:
        cache = {}
        end = len(nums) - 1

        def subproblem(start):
            if start in cache:
                return cache[start]
            
            if (end - start) == 0:
                cache[start] = nums[start]
            
            elif (end - start) == 1:
                cache[start] = max(nums[start], nums[end])
            
            else:
                cache[start] = max(
                    nums[start] + subproblem(start + 2),
                    subproblem(start + 1)
                )
            return cache[start]

        return subproblem(0)

# Summary - Buttom Up Iterative Approach
We will process up the chain.

At each element, we record the maximum robbed money value by this point.

So at the start, it will just be the element itself.

Then at the second element, we either just take the previous, or we skip the previous and note down the second element's value, whichever is bigger.

At the third, we make the same choice again, we either add the current value to the previous previous position's max, or simply take the previous position's max, again taking whichever is the largest to establish the maximum value by this position.

We can see a pattern, it's always add current value to the previous previous max value, or the previous max value. 

We can iteratively do this until we reach the last element of the array

## Time Complexity
$O(n)$ because we are iterating through every element of `nums` of length `n`

## Space Complexity
In the full array version it would be $O(n)$ because `dp` is the same length as `nums`. But in the simplified solution it is O(1) because we are always only keeping two additional variables at all time.

In [16]:
from typing import List

class SolutionFullArray:
    def rob(self, nums: List[int]) -> int:
        if len(nums) == 1:
            return nums[0]

        if len(nums) == 2:
            return max(nums[0], nums[1])
        
        # dp = [None] * len(nums)
        dp = []
        dp.append(nums[0])
        dp.append(max(dp[0], nums[1]))

        for i in range(2, len(nums)):
            dp.append(max(dp[i - 2] + nums[i], dp[i - 1]))
        return dp[-1]

In [ ]:
from typing import List

class Solution:
    def rob(self, nums: List[int]) -> int:
        if len(nums) == 1:
            return nums[0]

        if len(nums) == 2:
            return max(nums[0], nums[1])
        
        # dp = [None] * len(nums)
        dp = [nums[0], max(nums[0], nums[1])]

        for i in range(2, len(nums)):
            prevprev = dp[0]
            prev = dp[1]

            # then preprev will die off, current prev will
            # become prevprev (move to index 0)
            # dp[1] will then be current max

            dp[0] = prev
            dp[1] = max(nums[i] + prevprev, prev)
        return dp[-1]

# Debug

In [ ]:
from typing import List


class SolutionDebug:
    def rob(self, nums: List[int]) -> int:
        cache = {}

        def subproblem(start, end, cache):
            print(f"Processing subproblem({start}, {end}, {cache})")
            if (start, end) in cache:
                print(f"({start}, {end}) already in {cache}")
                return cache[(start, end)]
            
            if (end - start) == 0:
                print(f"{start} - {end} == 0")
                cache[(start, end)] = nums[start]
            
            elif (end - start) == 1:
                print(f"{start} - {end} == 1")
                cache[(start, end)] = max(nums[start], nums[end])
            
            else:
                print(f"Base case not reached, calculating subproblem({start + 2}, {end}) and subproblem({start + 1}, {end})")
                cache[(start, end)] = max(
                    nums[start] + subproblem(start + 2, end, cache),
                    subproblem(start + 1, end, cache)
                )
            print(f"Returning cache[({start}, {end})] = {cache[(start, end)]}")
            return cache[(start, end)]

        return subproblem(0, len(nums) - 1, {})

In [14]:
nums = [1,2,3,1]
s = Solution()

s.rob(nums)

Processing subproblem(0, 3, {})
Base case not reached, calculating subproblem(2, 3) and subproblem(1, 3)
Processing subproblem(2, 3, {})
2 - 3 == 1
Returning cache[(2, 3)] = 3
Processing subproblem(1, 3, {(2, 3): 3})
Base case not reached, calculating subproblem(3, 3) and subproblem(2, 3)
Processing subproblem(3, 3, {(2, 3): 3})
3 - 3 == 0
Returning cache[(3, 3)] = 1
Processing subproblem(2, 3, {(2, 3): 3, (3, 3): 1})
(2, 3) already in {(2, 3): 3, (3, 3): 1}
Returning cache[(1, 3)] = 3
Returning cache[(0, 3)] = 4


4